#LP-1 Laboratory
#High Performance Computing Assignment-1
###Name: Pratik Agarwal
###Roll No: 41103
###Batch: P1

##Problem Statement-
####a)Implement Parallel Reduction using Min, Max, Sum and Average operations.
####b) Write a CUDA program that, given an N-element vector, find-
####•The maximum element in the vector
####•The minimum element in the vector
####•The arithmetic mean of the vector
####•The standard deviation of the values in the vector
####Test for input N and generate a randomized vector V of length N (N should be large). The program should generate output as the two computed maximum values as well as the time taken to find each value.



In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-fs-prebuilt' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'


In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2021-08-14 15:49:57--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.16.29
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.16.29|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2021-08-14 15:49:57--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?_DrSPzygoJ1TyoNrNM28-ExmTcbb0xKMK2azQxmds5AhR940noVrFUNGCaxn8ps22widTOEL4DBEKo8E2yQjVr_ieuMEa15fHON6tbk_MyeghGF

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-lpbuked9
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-lpbuked9
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=3775f1b25b96a5e1a27617caa77d4cce653ea44ddcdd5d0099b12b8a81ca05b5
  Stored in directory: /tmp/pip-ephem-wheel-cache-bu997q2t/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu

#include<iostream>
#include<math.h>
using namespace std;

__global__ void maximum(int* arr2){
    int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;
    
    while(number_of_threads>0) {
        if(tid < number_of_threads) {
            int first = tid*step_size*2;
            int second = first + step_size;
            if(arr2[second] > arr2[first])
              arr2[first] = arr2[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
    }
}

__global__ void minimum(int *input) {
    int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;
    
    printf("No of threads = %d", number_of_threads);
    
    while(number_of_threads>0) {
        if(tid < number_of_threads) {
            int first = tid*step_size*2;
            int second = first + step_size;
            if(input[second] < input[first])
              input[first] = input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
    }
}

__global__ void addition(int* arr2) {
    const int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;
    
    while(number_of_threads > 0) {
        if(tid < number_of_threads) {
            int first = tid * step_size * 2;
            int second = first + step_size;
            
            arr2[first] += arr2[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
       
    }
}



int main(){
    int n = 10;                     //size of array
    int *arr, *arr2;
    int size = n*sizeof(int);

    arr = (int*)malloc(size);

    for (int i=0;i<n;i++){          //generating array of random numbers
        arr[i] = rand()%100;
    }
    
    cout<<"Vector = [";
    for (int i=0;i<n;i++){          //displaying array
        cout<<arr[i]<<", ";
    }
    cout<<"]";

    int min, max, sum;
    float mean;

    cudaMalloc((void**)&arr2, size);
    
    cudaMemcpy(arr2, arr, size, cudaMemcpyHostToDevice);

    maximum<<<1, n/2>>>(arr2);
    cudaMemcpy(&max, arr2, sizeof(int), cudaMemcpyDeviceToHost);
    cout<<"\nMaximum element = "<<max<<endl;

    minimum<<<1, n/2>>>(arr2);
    cudaMemcpy(&min, arr2, sizeof(int), cudaMemcpyDeviceToHost);
    cout<<"\nMinimum element = "<<min<<endl;

    addition<<<1, n/2>>>(arr2);
    cudaMemcpy(&sum, arr2, sizeof(int), cudaMemcpyDeviceToHost);
    cout<<"\nAddition = "<<sum<<endl;

    mean = (float)sum/n;
    cout<<"\nAverage = "<<mean<<endl;
    
    return 0;
}

Vector = [83, 86, 77, 15, 93, 35, 86, 92, 49, 21, ]
Maximum element = 93
No of threads = 5No of threads = 5No of threads = 5No of threads = 5No of threads = 5
Minimum element = 15

Addition = 385

Average = 38.5



####b) Write a CUDA program that, given an N-element vector, find-
####•The maximum element in the vector
####•The minimum element in the vector
####•The arithmetic mean of the vector
####•The standard deviation of the values in the vector

In [ ]:
code = """

#include<iostream>
#include<math.h>
using namespace std;

__global__ void maximum(int* arr2, int n){
    int m = arr2[0];
    for(int i=0;i<n;i++){
      if(m<arr2[i]){
        m = arr2[i];
      }
    }
    arr2[0] =  m;
}

__global__ void minimum(int* arr2, int n){
    int m = arr2[0];
    for(int i=0;i<n;i++){
      if(m>arr2[i]){
        m = arr2[i];
      }
    }
    arr2[0] =  m;
}

__global__ void addition(int* arr2) {
    const int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;
    
    while(number_of_threads > 0) {
        if(tid < number_of_threads) {
            int first = tid * step_size * 2;
            int second = first + step_size;
            
            arr2[first] += arr2[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
       
    }
}

__global__ void mean_diff_sq(float *input, float mean) {
    input[threadIdx.x] -= mean;
    input[threadIdx.x] *= input[threadIdx.x];
}

__global__ void sum_floats(float *input) {
    int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;
    
    while(number_of_threads > 0) {
        if(tid < number_of_threads) {
            int first = tid * step_size * 2;
            int second = first + step_size;
            
            input[first] += input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
       
    }
}

void copy_int_to_float(float *dest, int *src, int size){
    for(int i=0; i<size; i++)
        dest[i] = float(src[i]);
}


int main(){
    int n = 50;                     //size of array
    int *arr, *arr2;
    int size = n*sizeof(int);

    arr = (int*)malloc(size);

    for (int i=0;i<n;i++){          //generating array of random numbers
        arr[i] = rand()%1000;
    }
    
    cout<<"Vector = [";
    for (int i=0;i<n;i++){          //displaying array
        cout<<arr[i]<<", ";
    }
    cout<<"]"<<endl;

    int min, max;
    long sum;
    float mean, stdval;

    cudaMalloc((void**)&arr2, size);
    cudaMemcpy(arr2, arr, size, cudaMemcpyHostToDevice);

    maximum<<<1, n/2>>>(arr2, n);
    cudaMemcpy(&max, arr2, sizeof(int), cudaMemcpyDeviceToHost);
    cout<<"Maximum element = "<<max<<endl;

    minimum<<<1, n/2>>>(arr2, n);
    cudaMemcpy(&min, arr2, sizeof(int), cudaMemcpyDeviceToHost);
    cout<<"Minimum element = "<<min<<endl;

    addition<<<1, n/2>>>(arr2);
    cudaMemcpy(&sum, arr2, sizeof(int), cudaMemcpyDeviceToHost);
    cout<<"Addition = "<<sum<<endl;
    mean = (float)sum/n;
    cout<<"Average = "<<mean<<endl;
    


    float *arr_float;
    float *arr_std;
    
    arr_float = (float *)malloc(n*sizeof(float));
    cudaMalloc((void **)&arr_std, n*sizeof(float));
    
    copy_int_to_float(arr_float, arr, n);
    
    cudaMemcpy(arr_std, arr_float, n*sizeof(float), cudaMemcpyHostToDevice);
    
    mean_diff_sq <<<1,n>>>(arr_std, mean);
    sum_floats<<<1,n/2>>>(arr_std);
    
    cudaMemcpy(&stdval, arr_std, sizeof(float), cudaMemcpyDeviceToHost);


    stdval = stdval / n;
    stdval = sqrt(stdval);
    
    cout<<"Standard deviation = "<<stdval<<endl;
    
    return 0;
}"""

In [ ]:
myfile = open("HPC_Ass1.cu", "w")
myfile.write(code)
myfile.close()

In [ ]:
!nvcc HPC_Ass1.cu

!./a.out

Vector = [383, 886, 777, 915, 793, 335, 386, 492, 649, 421, 362, 27, 690, 59, 763, 926, 540, 426, 172, 736, 211, 368, 567, 429, 782, 530, 862, 123, 67, 135, 929, 802, 22, 58, 69, 167, 393, 456, 11, 42, 229, 373, 421, 919, 784, 537, 198, 324, 315, 370, ]
Maximum element = 929
Minimum element = 11
Addition = 16171
Average = 323.42
Standard deviation = 272.402
